In [15]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from scipy.stats import gaussian_kde
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tsa.stattools import acf
import warnings
import os
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 1. Carga y Preprocesamiento de Datos

In [16]:
# Cargar datos
base = Path('.')
candidates = list(base.glob('**/Tasa*colombiano*.*'))
if not candidates:
    raise FileNotFoundError('No se encontró el archivo de tasa de cambio')
file_path = candidates[0]
print(f'📂 Usando archivo: {file_path}')

if file_path.suffix.lower() in ('.xls', '.xlsx'):
    df = pd.read_excel(file_path)
else:
    df = pd.read_csv(file_path, encoding='latin1', sep=';')

print(f"✅ Datos cargados: {df.shape}")

📂 Usando archivo: Tasa de cambio del peso colombiano .xlsx
✅ Datos cargados: (8862, 18)
✅ Datos cargados: (8862, 18)


In [17]:
# Seleccionar últimos 730 días (2 años)
df_clean = df.iloc[8857-730:8857].reset_index(drop=True)

# Preparar datos
df_preprocessed = df_clean[['periodo', 'trm']].copy()

# Calcular primera diferencia
df_preprocessed['y_t'] = df_preprocessed['trm'].diff()

print(f"📊 Datos preprocesados: {len(df_preprocessed)} registros")
print(f"📅 Periodo: {df_preprocessed['periodo'].min()} a {df_preprocessed['periodo'].max()}")
display(df_preprocessed.head(10))

📊 Datos preprocesados: 730 registros
📅 Periodo: 2023-01-20 00:00:00 a 2025-11-06 00:00:00


,periodo,trm,y_t
0,2023-01-20,4683.85,NaN
1,2023-01-23,4631.64,-52.21
2,2023-01-24,4551.02,-80.62
3,2023-01-25,4545.94,-5.08
4,2023-01-26,4538.91,-7.03
5,2023-01-27,4531.75,-7.16
6,2023-01-30,4548.50,16.75
7,2023-01-31,4632.20,83.70
8,2023-02-01,4648.70,16.50
9,2023-02-02,4639.04,-9.66


## 2. Función para Crear Características (AR + MA)

In [18]:
def create_features(df, n_ar_lags=3, n_ma_windows=[2, 3]):
    """
    Crea características AR (autorregresivos) y MA (promedios móviles).
    
    Args:
        df: DataFrame con columna 'y_t'
        n_ar_lags: Número de lags AR a crear
        n_ma_windows: Lista de ventanas para MA
    
    Returns:
        DataFrame con features, lista de nombres de features
    """
    df_features = df.copy()
    feature_names = []
    
    # AR Lags
    for lag in range(1, n_ar_lags + 1):
        col_name = f'y_t-{lag}'
        df_features[col_name] = df_features['y_t'].shift(lag)
        feature_names.append(col_name)
    
    # MA Windows
    for window in n_ma_windows:
        col_name = f'm_{window}'
        df_features[col_name] = df_features['y_t'].rolling(window=window, min_periods=window).mean()
        feature_names.append(col_name)
    
    return df_features, feature_names

print("✅ Función create_features definida")

✅ Función create_features definida


## 3. Implementación de ANFIS (Adaptive Neuro-Fuzzy Inference System)

In [19]:
class ANFIS:
    """
    Adaptive Neuro-Fuzzy Inference System (ANFIS)
    Implementación Takagi-Sugeno de orden 1
    
    Arquitectura de 5 capas:
    1. Fuzzificación (Gaussiana)
    2. Reglas difusas (T-norma producto)
    3. Normalización
    4. Consecuentes (lineales)
    5. Defuzzificación (suma ponderada)
    """
    
    def __init__(self, n_inputs, n_rules, mf_type='gaussian', sigma_init=1.0, random_state=42):
        np.random.seed(random_state)
        self.n_inputs = n_inputs
        self.n_rules = n_rules
        self.mf_type = mf_type
        self.sigma_init = sigma_init
        
        # Parámetros de las funciones de membresía (premise)
        self.centers = np.random.randn(n_inputs, n_rules) * 0.5
        self.sigmas = np.ones((n_inputs, n_rules)) * sigma_init
        
        # Parámetros de los consecuentes (consequence)
        self.consequents = np.random.randn(n_rules, n_inputs + 1) * 0.01
        
        self.training_history = []
    
    def gaussian_mf(self, x, center, sigma):
        """Función de membresía Gaussiana"""
        return np.exp(-0.5 * ((x - center) / (sigma + 1e-10))**2)
    
    def triangular_mf(self, x, left, center, right):
        """Función de membresía Triangular"""
        result = np.zeros_like(x)
        mask1 = (x >= left) & (x < center)
        mask2 = (x >= center) & (x <= right)
        result[mask1] = (x[mask1] - left) / (center - left + 1e-10)
        result[mask2] = (right - x[mask2]) / (right - center + 1e-10)
        return result
    
    def forward(self, X):
        """
        Forward pass a través de las 5 capas de ANFIS
        """
        n_samples = X.shape[0]
        
        # Capa 1: Fuzzificación
        memberships = np.zeros((n_samples, self.n_inputs, self.n_rules))
        for i in range(self.n_inputs):
            for j in range(self.n_rules):
                if self.mf_type == 'gaussian':
                    memberships[:, i, j] = self.gaussian_mf(
                        X[:, i], 
                        self.centers[i, j], 
                        self.sigmas[i, j]
                    )
        
        # Capa 2: Reglas difusas (T-norma producto)
        firing_strengths = np.prod(memberships, axis=1)  # (n_samples, n_rules)
        
        # Evitar división por cero
        firing_strengths = np.clip(firing_strengths, 1e-10, None)
        
        # Capa 3: Normalización
        firing_sum = firing_strengths.sum(axis=1, keepdims=True) + 1e-10
        normalized_firing = firing_strengths / firing_sum
        
        # Capa 4: Consecuentes lineales
        X_extended = np.column_stack([X, np.ones(n_samples)])
        rule_outputs = X_extended @ self.consequents.T  # (n_samples, n_rules)
        
        # Capa 5: Defuzzificación (suma ponderada)
        output = np.sum(normalized_firing * rule_outputs, axis=1)
        
        return output, normalized_firing, rule_outputs
    
    def fit(self, X, y, epochs=100, lr_premise=0.01, lr_consequent=0.1, 
            reg_lambda=0.01, early_stopping_patience=20, verbose=False):
        """
        Entrenar ANFIS usando descenso de gradiente
        """
        n_samples = X.shape[0]
        best_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(epochs):
            # Forward pass
            y_pred, normalized_firing, rule_outputs = self.forward(X)
            
            # Error
            errors = y - y_pred
            mse = np.mean(errors**2)
            mae = np.mean(np.abs(errors))
            
            # Regularización L2 en consecuentes
            reg_loss = reg_lambda * np.sum(self.consequents**2)
            total_loss = mse + reg_loss
            
            self.training_history.append({'epoch': epoch, 'mse': mse, 'mae': mae})
            
            # Early stopping
            if total_loss < best_loss:
                best_loss = total_loss
                patience_counter = 0
            else:
                patience_counter += 1
            
            if patience_counter >= early_stopping_patience:
                if verbose:
                    print(f"\nEarly stopping en época {epoch}")
                break
            
            # Actualizar consecuentes usando mínimos cuadrados regularizados
            X_extended = np.column_stack([X, np.ones(n_samples)])
            for j in range(self.n_rules):
                W = np.diag(normalized_firing[:, j])
                XTW = X_extended.T @ W
                
                # Regularización Ridge
                reg_matrix = reg_lambda * np.eye(self.n_inputs + 1)
                
                try:
                    self.consequents[j] = np.linalg.solve(
                        XTW @ X_extended + reg_matrix,
                        XTW @ y
                    )
                except np.linalg.LinAlgError:
                    pass
            
            # Actualizar parámetros de premise (centers y sigmas)
            if epoch % 5 == 0:  # Actualizar cada 5 épocas para estabilidad
                for i in range(self.n_inputs):
                    for j in range(self.n_rules):
                        # Gradiente respecto a centers
                        diff = X[:, i] - self.centers[i, j]
                        grad_center = np.mean(errors * normalized_firing[:, j] * 
                                             rule_outputs[:, j] * diff / (self.sigmas[i, j]**2 + 1e-10))
                        self.centers[i, j] += lr_premise * grad_center
                        
                        # Gradiente respecto a sigmas
                        grad_sigma = np.mean(errors * normalized_firing[:, j] * 
                                            rule_outputs[:, j] * (diff**2) / (self.sigmas[i, j]**3 + 1e-10))
                        self.sigmas[i, j] += lr_premise * grad_sigma
                        self.sigmas[i, j] = max(0.1, self.sigmas[i, j])  # Evitar sigmas muy pequeños
            
            if verbose and epoch % 50 == 0:
                print(f"Epoch {epoch}/{epochs} - MSE: {mse:.6f}, MAE: {mae:.6f}")
        
        return self
    
    def predict(self, X):
        """Predicción usando el modelo entrenado"""
        output, _, _ = self.forward(X)
        return output

print("✅ Clase ANFIS implementada correctamente")

✅ Clase ANFIS implementada correctamente


## 4. Grid Search para ANFIS

In [20]:
# Definir espacio de búsqueda
PARAM_GRID = {
    'n_mf': [2, 3, 4],              # Número de funciones de membresía por entrada
    'n_ar_lags': [2, 3, 4, 5],      # Lags autorregresivos
    'n_ma_windows': [[2], [2,3], [2,3,4]],  # Ventanas de promedio móvil
    'reg_lambda': [0.001, 0.01, 0.1],  # Regularización
    'mf_type': ['gaussian']  # Tipo de función de membresía
}

print("📋 CONFIGURACIÓN DEL GRID SEARCH")
print("=" * 60)
for param, values in PARAM_GRID.items():
    print(f"  {param:<15}: {values}")

total_combinations = (len(PARAM_GRID['n_mf']) * 
                     len(PARAM_GRID['n_ar_lags']) * 
                     len(PARAM_GRID['n_ma_windows']) * 
                     len(PARAM_GRID['reg_lambda']) *
                     len(PARAM_GRID['mf_type']))
print(f"\n  Total de combinaciones: {total_combinations}")
print("=" * 60)

📋 CONFIGURACIÓN DEL GRID SEARCH
  n_mf           : [2, 3, 4]
  n_ar_lags      : [2, 3, 4, 5]
  n_ma_windows   : [[2], [2, 3], [2, 3, 4]]
  reg_lambda     : [0.001, 0.01, 0.1]
  mf_type        : ['gaussian']

  Total de combinaciones: 108


In [21]:
# Función para evaluar una configuración
def evaluate_config(df_preprocessed, n_mf, n_ar_lags, n_ma_windows, reg_lambda, mf_type):
    """
    Evalúa una configuración específica de ANFIS.
    Sin partición de datos - usa todos los datos para entrenamiento.
    """
    # Crear features
    df_features, feature_names = create_features(
        df_preprocessed,
        n_ar_lags=n_ar_lags,
        n_ma_windows=n_ma_windows
    )
    
    # Eliminar NaN
    df_clean = df_features.dropna().reset_index(drop=True)
    
    # Preparar X, y
    X = df_clean[feature_names].values
    y = df_clean['y_t'].values
    
    if len(X) < 50:  # Validación mínima de datos
        return None
    
    # Entrenar modelo ANFIS
    model = ANFIS(
        n_inputs=X.shape[1],
        n_rules=n_mf,
        mf_type=mf_type,
        sigma_init=1.0,
        random_state=42
    )
    
    model.fit(
        X, y,
        epochs=300,
        lr_premise=0.01,
        lr_consequent=0.1,
        reg_lambda=reg_lambda,
        early_stopping_patience=30,
        verbose=False
    )
    
    # Predicción
    y_pred = model.predict(X)
    
    # Métricas
    mse = np.mean((y - y_pred)**2)
    mae = np.mean(np.abs(y - y_pred))
    rmse = np.sqrt(mse)
    
    # R² score
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = 1 - (ss_res / (ss_tot + 1e-10))
    
    return {
        'n_mf': n_mf,
        'n_ar_lags': n_ar_lags,
        'n_ma_windows': len(n_ma_windows),
        'reg_lambda': reg_lambda,
        'mf_type': mf_type,
        'mse': mse,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'n_samples': len(X),
        'n_features': X.shape[1],
        'feature_names': feature_names,
        'model': model,
        'X': X,
        'y': y,
        'y_pred': y_pred
    }

print("✅ Función evaluate_config definida")

✅ Función evaluate_config definida


In [22]:
# Ejecutar Grid Search
import time
from itertools import product

print("\n" + "=" * 70)
print("🔍 INICIANDO GRID SEARCH")
print("=" * 70)

results = []
start_time = time.time()

# Generar todas las combinaciones
param_combinations = list(product(
    PARAM_GRID['n_mf'],
    PARAM_GRID['n_ar_lags'],
    PARAM_GRID['n_ma_windows'],
    PARAM_GRID['reg_lambda'],
    PARAM_GRID['mf_type']
))

total = len(param_combinations)

for i, (n_mf, n_ar, n_ma_windows, reg_lambda, mf_type) in enumerate(param_combinations, 1):
    print(f"\r[{i}/{total}] Probando: n_mf={n_mf}, AR={n_ar}, MA_win={len(n_ma_windows)}, reg={reg_lambda:.3f}", end="")
    
    result = evaluate_config(
        df_preprocessed,
        n_mf=n_mf,
        n_ar_lags=n_ar,
        n_ma_windows=n_ma_windows,
        reg_lambda=reg_lambda,
        mf_type=mf_type
    )
    
    if result is not None:
        results.append(result)

elapsed = time.time() - start_time
print(f"\n\n✅ Grid Search completado en {elapsed:.2f} segundos")
print(f"📊 Configuraciones evaluadas: {len(results)}/{total}")


🔍 INICIANDO GRID SEARCH
[108/108] Probando: n_mf=4, AR=5, MA_win=3, reg=0.100

✅ Grid Search completado en 33.02 segundos
📊 Configuraciones evaluadas: 108/108


✅ Grid Search completado en 33.02 segundos
📊 Configuraciones evaluadas: 108/108


In [23]:
# Analizar resultados del Grid Search
results_df = pd.DataFrame([{
    'n_mf': r['n_mf'],
    'n_ar_lags': r['n_ar_lags'],
    'n_ma_windows': r['n_ma_windows'],
    'reg_lambda': r['reg_lambda'],
    'mf_type': r['mf_type'],
    'mse': r['mse'],
    'mae': r['mae'],
    'rmse': r['rmse'],
    'r2': r['r2'],
    'n_samples': r['n_samples'],
    'n_features': r['n_features']
} for r in results])

# Ordenar por MSE (menor es mejor)
results_df = results_df.sort_values('mse').reset_index(drop=True)

print("\n" + "=" * 70)
print("🏆 TOP 10 MEJORES CONFIGURACIONES (por MSE)")
print("=" * 70)
display(results_df.head(10))

# Guardar resultados
results_df.to_csv('ANFIS/grid_search_results_completo.csv', index=False)
print("\n💾 Resultados guardados en: ANFIS/grid_search_results_completo.csv")


🏆 TOP 10 MEJORES CONFIGURACIONES (por MSE)


,n_mf,n_ar_lags,n_ma_windows,reg_lambda,mf_type,mse,mae,rmse,r2,n_samples,n_features
0,2,4,3,0.001,gaussian,1.096185e-13,2.487227e-07,3.310869e-07,1.0,725,7
1,2,3,3,0.001,gaussian,1.226264e-13,2.530699e-07,3.501806e-07,1.0,726,6
2,2,2,2,0.001,gaussian,1.283061e-13,2.688208e-07,3.581985e-07,1.0,727,4
3,2,3,2,0.001,gaussian,1.289180e-13,2.696901e-07,3.590515e-07,1.0,726,5
4,2,4,2,0.001,gaussian,1.431217e-13,2.790321e-07,3.783143e-07,1.0,725,6
5,2,2,3,0.001,gaussian,1.700961e-13,3.062529e-07,4.124270e-07,1.0,726,5
6,2,3,1,0.001,gaussian,2.101328e-13,3.435316e-07,4.584025e-07,1.0,726,4
7,2,2,1,0.001,gaussian,2.342705e-13,3.463381e-07,4.840149e-07,1.0,727,3
8,3,3,3,0.001,gaussian,2.449844e-13,3.717440e-07,4.949590e-07,1.0,726,6
9,2,5,1,0.001,gaussian,2.522453e-13,3.710419e-07,5.022403e-07,1.0,724,6



💾 Resultados guardados en: ANFIS/grid_search_results_completo.csv


## 5. Entrenar Modelo Final con Mejor Configuración

In [24]:
# Obtener mejor configuración
best_idx = results_df['mse'].idxmin()
best_result = results[best_idx]

print("=" * 70)
print("🏆 MEJOR CONFIGURACIÓN ENCONTRADA")
print("=" * 70)
print(f"\nParámetros:")
print(f"  • Funciones de Membresía (n_mf): {best_result['n_mf']}")
print(f"  • Lags AR: {best_result['n_ar_lags']}")
print(f"  • Ventanas MA: {best_result['n_ma_windows']}")
print(f"  • Regularización (lambda): {best_result['reg_lambda']}")
print(f"  • Tipo de MF: {best_result['mf_type']}")
print(f"\nMétricas:")
print(f"  • MSE: {best_result['mse']:.6f}")
print(f"  • MAE: {best_result['mae']:.6f}")
print(f"  • RMSE: {best_result['rmse']:.6f}")
print(f"  • R²: {best_result['r2']:.6f}")
print(f"\nDatos:")
print(f"  • Muestras: {best_result['n_samples']}")
print(f"  • Features: {best_result['n_features']}")
print(f"  • Nombres: {best_result['feature_names']}")
print("=" * 70)

# Extraer datos del mejor modelo
best_model = best_result['model']
X_best = best_result['X']
y_best = best_result['y']
y_pred_best = best_result['y_pred']
residuals = y_best - y_pred_best

🏆 MEJOR CONFIGURACIÓN ENCONTRADA

Parámetros:
  • Funciones de Membresía (n_mf): 2
  • Lags AR: 2
  • Ventanas MA: 1
  • Regularización (lambda): 0.001
  • Tipo de MF: gaussian

Métricas:
  • MSE: 0.000000
  • MAE: 0.000000
  • RMSE: 0.000000
  • R²: 1.000000

Datos:
  • Muestras: 727
  • Features: 3
  • Nombres: ['y_t-1', 'y_t-2', 'm_2']


## 6. Análisis de Residuales

In [25]:
# Estadísticas de residuales
from statsmodels.stats.stattools import durbin_watson

print("=" * 70)
print("📊 ANÁLISIS DE RESIDUALES")
print("=" * 70)

# 1. Media cero
mean_res = np.mean(residuals)
std_res = np.std(residuals)
print(f"\n1️⃣ MEDIA CERO:")
print(f"   Media: {mean_res:.6f}")
print(f"   {'✅ CUMPLE' if abs(mean_res) < 0.1 else '❌ NO CUMPLE'}")

# 2. Normalidad
shapiro_stat, shapiro_p = stats.shapiro(residuals)
jb_stat, jb_p = stats.jarque_bera(residuals)
print(f"\n2️⃣ NORMALIDAD:")
print(f"   Shapiro-Wilk p-value: {shapiro_p:.4f}")
print(f"   Jarque-Bera p-value: {jb_p:.4f}")
print(f"   {'✅ CUMPLE' if shapiro_p > 0.05 else '⚠️ NO CUMPLE (común en series financieras)'}")

# 3. No autocorrelación
dw_stat = durbin_watson(residuals)
autocorr_lag1 = np.corrcoef(residuals[:-1], residuals[1:])[0, 1]
print(f"\n3️⃣ NO AUTOCORRELACIÓN:")
print(f"   Durbin-Watson: {dw_stat:.4f} (ideal: 2.0)")
print(f"   Autocorr Lag-1: {autocorr_lag1:.4f}")
print(f"   {'✅ CUMPLE' if 1.5 < dw_stat < 2.5 else '❌ NO CUMPLE'}")

# 4. Homocedasticidad
n_half = len(residuals) // 2
var_first_half = np.var(residuals[:n_half])
var_second_half = np.var(residuals[n_half:])
var_ratio = max(var_first_half, var_second_half) / min(var_first_half, var_second_half)
print(f"\n4️⃣ HOMOCEDASTICIDAD:")
print(f"   Ratio de varianzas: {var_ratio:.4f}")
print(f"   {'✅ CUMPLE' if var_ratio < 2 else '❌ NO CUMPLE'}")

print("\n" + "=" * 70)

📊 ANÁLISIS DE RESIDUALES

1️⃣ MEDIA CERO:
   Media: 0.000000
   ✅ CUMPLE

2️⃣ NORMALIDAD:
   Shapiro-Wilk p-value: 0.0000
   Jarque-Bera p-value: 0.0000
   ⚠️ NO CUMPLE (común en series financieras)

3️⃣ NO AUTOCORRELACIÓN:
   Durbin-Watson: 2.2679 (ideal: 2.0)
   Autocorr Lag-1: -0.1355
   ✅ CUMPLE

4️⃣ HOMOCEDASTICIDAD:
   Ratio de varianzas: 1.4590
   ✅ CUMPLE



## 7. Generación de 6 Gráficas de Diagnóstico

In [26]:
# Crear carpeta para gráficas
os.makedirs('ANFIS', exist_ok=True)

print("=" * 70)
print("📊 GENERANDO 6 GRÁFICAS DE DIAGNÓSTICO")
print("=" * 70)

# 1. QQ-PLOT
fig1, ax1 = plt.subplots(figsize=(10, 8))
stats.probplot(residuals, dist="norm", plot=ax1)
ax1.set_title('QQ-Plot de Residuos\n(Comparación con Distribución Normal)', 
              fontsize=14, fontweight='bold')
ax1.set_xlabel('Cuantiles Teóricos', fontsize=12)
ax1.set_ylabel('Cuantiles Observados', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.text(0.05, 0.95, f'Shapiro-Wilk p={shapiro_p:.4f}', 
         transform=ax1.transAxes, fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
fig1.savefig('ANFIS/1_qqplot_residuos.png', dpi=300, bbox_inches='tight')
plt.close(fig1)
print("✅ 1. QQ-Plot guardado")

# 2. PP-PLOT
fig2, ax2 = plt.subplots(figsize=(10, 8))
residuals_sorted = np.sort(residuals)
n = len(residuals_sorted)
prob_empirica = np.arange(1, n + 1) / (n + 1)
prob_teorica = stats.norm.cdf(residuals_sorted, loc=mean_res, scale=std_res)
ax2.scatter(prob_teorica, prob_empirica, alpha=0.6, s=30, 
            edgecolors='navy', facecolors='royalblue')
ax2.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Línea ideal')
ax2.set_title('P-P Plot de Residuos\n(Probabilidad Acumulada)', 
              fontsize=14, fontweight='bold')
ax2.set_xlabel('Probabilidad Teórica (Normal)', fontsize=12)
ax2.set_ylabel('Probabilidad Empírica', fontsize=12)
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(-0.05, 1.05)
ax2.set_ylim(-0.05, 1.05)
plt.tight_layout()
fig2.savefig('ANFIS/2_ppplot_residuos.png', dpi=300, bbox_inches='tight')
plt.close(fig2)
print("✅ 2. P-P Plot guardado")

# 3. VALORES REALES VS PREDICCIÓN
fig3, ax3 = plt.subplots(figsize=(14, 8))
indices = np.arange(len(y_best))
ax3.plot(indices, y_best, 'b-', linewidth=1.5, label='Valores Reales', alpha=0.8)
ax3.plot(indices, y_pred_best, 'r--', linewidth=1.5, label='Predicción ANFIS', alpha=0.8)
ax3.fill_between(indices, y_best, y_pred_best, alpha=0.2, color='gray')
ax3.set_title('Valores Reales vs Predicción ANFIS\n(Serie Temporal Completa)', 
              fontsize=14, fontweight='bold')
ax3.set_xlabel('Índice Temporal', fontsize=12)
ax3.set_ylabel('y_t (Primera Diferencia TRM)', fontsize=12)
ax3.legend(loc='upper right', fontsize=11)
ax3.grid(True, alpha=0.3)
textstr = f'R² = {best_result["r2"]:.4f}\nRMSE = {best_result["rmse"]:.4f}\nMAE = {best_result["mae"]:.4f}'
ax3.text(0.02, 0.98, textstr, transform=ax3.transAxes, fontsize=10,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
fig3.savefig('ANFIS/3_real_vs_prediccion.png', dpi=300, bbox_inches='tight')
plt.close(fig3)
print("✅ 3. Real vs Predicción guardado")

# 4. DISTRIBUCIÓN DEL RESIDUAL
fig4, ax4 = plt.subplots(figsize=(10, 8))
n_bins = 40
ax4.hist(residuals, bins=n_bins, density=True, alpha=0.7, 
         color='steelblue', edgecolor='white', linewidth=0.5)
kde = gaussian_kde(residuals)
x_kde = np.linspace(residuals.min() - 0.5, residuals.max() + 0.5, 200)
ax4.plot(x_kde, kde(x_kde), 'darkblue', linewidth=2, label='KDE')
x_norm = np.linspace(residuals.min() - 0.5, residuals.max() + 0.5, 200)
y_norm = stats.norm.pdf(x_norm, mean_res, std_res)
ax4.plot(x_norm, y_norm, 'r--', linewidth=2, label='Normal Teórica')
ax4.axvline(x=0, color='green', linestyle=':', linewidth=2, label='Media = 0')
ax4.axvline(x=mean_res, color='orange', linestyle='-', linewidth=2, 
            label=f'Media Real = {mean_res:.3f}')
ax4.set_title('Distribución de Residuos\n(Histograma + Densidad)', 
              fontsize=14, fontweight='bold')
ax4.set_xlabel('Residuo', fontsize=12)
ax4.set_ylabel('Densidad', fontsize=12)
ax4.legend(loc='upper right')
ax4.grid(True, alpha=0.3)
textstr = f'μ = {mean_res:.4f}\nσ = {std_res:.4f}\nSkew = {stats.skew(residuals):.4f}\nKurt = {stats.kurtosis(residuals):.4f}'
ax4.text(0.02, 0.98, textstr, transform=ax4.transAxes, fontsize=10,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
fig4.savefig('ANFIS/4_distribucion_residuos.png', dpi=300, bbox_inches='tight')
plt.close(fig4)
print("✅ 4. Distribución de Residuos guardado")

# 5. GRÁFICA DEL RESIDUAL (Serie temporal)
fig5, ax5 = plt.subplots(figsize=(14, 6))
indices = np.arange(len(residuals))
ax5.plot(indices, residuals, 'b-', linewidth=0.8, alpha=0.7)
ax5.scatter(indices, residuals, c=residuals, cmap='RdYlGn_r', s=20, alpha=0.6, edgecolors='none')
ax5.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Residuo = 0')
ax5.axhline(y=2*std_res, color='orange', linestyle=':', linewidth=1.5, 
            label=f'±2σ = ±{2*std_res:.2f}')
ax5.axhline(y=-2*std_res, color='orange', linestyle=':', linewidth=1.5)
ax5.fill_between(indices, -2*std_res, 2*std_res, alpha=0.1, color='yellow')
ax5.set_title('Gráfica de Residuos vs Tiempo\n(Serie Temporal de Errores)', 
              fontsize=14, fontweight='bold')
ax5.set_xlabel('Índice Temporal', fontsize=12)
ax5.set_ylabel('Residuo (y_real - y_pred)', fontsize=12)
ax5.legend(loc='upper right')
ax5.grid(True, alpha=0.3)
outliers = np.sum(np.abs(residuals) > 2*std_res)
textstr = f'Outliers (|res|>2σ): {outliers}/{len(residuals)} ({100*outliers/len(residuals):.1f}%)'
ax5.text(0.02, 0.98, textstr, transform=ax5.transAxes, fontsize=10,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
fig5.savefig('ANFIS/5_grafica_residuos.png', dpi=300, bbox_inches='tight')
plt.close(fig5)
print("✅ 5. Gráfica de Residuos guardado")

# 6. AUTOCORRELACIÓN DE RESIDUALES (ACF)
fig6, ax6 = plt.subplots(figsize=(12, 6))
plot_acf(residuals, ax=ax6, lags=30, alpha=0.05)
ax6.set_title('Función de Autocorrelación (ACF) de Residuos\n(Verificación de Independencia)', 
              fontsize=14, fontweight='bold')
ax6.set_xlabel('Lag', fontsize=12)
ax6.set_ylabel('Autocorrelación', fontsize=12)
ax6.grid(True, alpha=0.3)
textstr = f'Durbin-Watson: {dw_stat:.4f}\nAutocorr Lag-1: {autocorr_lag1:.4f}'
ax6.text(0.75, 0.95, textstr, transform=ax6.transAxes, fontsize=10,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
fig6.savefig('ANFIS/6_autocorrelacion_residuos.png', dpi=300, bbox_inches='tight')
plt.close(fig6)
print("✅ 6. Autocorrelación de Residuos guardado")

print("\n" + "=" * 70)
print("📁 TODAS LAS GRÁFICAS GUARDADAS EN: ANFIS/")
print("=" * 70)
print("""
   1. 1_qqplot_residuos.png        - QQ-Plot (normalidad)
   2. 2_ppplot_residuos.png        - P-P Plot (probabilidad acumulada)
   3. 3_real_vs_prediccion.png     - Valores reales vs predicción
   4. 4_distribucion_residuos.png  - Histograma de residuos
   5. 5_grafica_residuos.png       - Serie temporal de residuos
   6. 6_autocorrelacion_residuos.png - ACF de residuos
""")
print("=" * 70)

📊 GENERANDO 6 GRÁFICAS DE DIAGNÓSTICO
✅ 1. QQ-Plot guardado
✅ 1. QQ-Plot guardado
✅ 2. P-P Plot guardado
✅ 2. P-P Plot guardado
✅ 3. Real vs Predicción guardado
✅ 3. Real vs Predicción guardado
✅ 4. Distribución de Residuos guardado
✅ 4. Distribución de Residuos guardado
✅ 5. Gráfica de Residuos guardado
✅ 5. Gráfica de Residuos guardado
✅ 6. Autocorrelación de Residuos guardado

📁 TODAS LAS GRÁFICAS GUARDADAS EN: ANFIS/

   1. 1_qqplot_residuos.png        - QQ-Plot (normalidad)
   2. 2_ppplot_residuos.png        - P-P Plot (probabilidad acumulada)
   3. 3_real_vs_prediccion.png     - Valores reales vs predicción
   4. 4_distribucion_residuos.png  - Histograma de residuos
   5. 5_grafica_residuos.png       - Serie temporal de residuos
   6. 6_autocorrelacion_residuos.png - ACF de residuos

✅ 6. Autocorrelación de Residuos guardado

📁 TODAS LAS GRÁFICAS GUARDADAS EN: ANFIS/

   1. 1_qqplot_residuos.png        - QQ-Plot (normalidad)
   2. 2_ppplot_residuos.png        - P-P Plot (probabil

## 8. Resumen Final del Modelo

In [29]:
print("\n" + "╔" + "═" * 78 + "╗")
print("║" + "" * 15 + "📊 RESUMEN FINAL DEL MODELO ANFIS" + " 30 ║")
print("╚" + "═" * 78 + "╝")

print("\n" + "━" * 80)
print("🏆 MEJOR CONFIGURACIÓN")
print("━" * 80)
print(f"""
   Parámetros del modelo:
   • Funciones de Membresía: {best_result['n_mf']}
   • Lags AR: {best_result['n_ar_lags']}
   • Ventanas MA: {best_result['n_ma_windows']}
   • Regularización: {best_result['reg_lambda']}
   • Tipo MF: {best_result['mf_type']}
   • Features: {best_result['feature_names']}
""")

print("\n" + "━" * 80)
print("📈 MÉTRICAS DE DESEMPEÑO")
print("━" * 80)
print(f"""
   • R² Score: {best_result['r2']:.6f}
   • MSE: {best_result['mse']:.6f}
   • MAE: {best_result['mae']:.6f}
   • RMSE: {best_result['rmse']:.6f}
   • Muestras: {best_result['n_samples']}
""")

print("\n" + "━" * 80)
print("✅ VERIFICACIÓN DE SUPUESTOS")
print("━" * 80)
print(f"""
   1. Media cero: {'✅ CUMPLE' if abs(mean_res) < 0.1 else '❌ NO CUMPLE'} (μ = {mean_res:.6f})
   2. Normalidad: {'✅ CUMPLE' if shapiro_p > 0.05 else '⚠️ NO CUMPLE'} (p = {shapiro_p:.4f})
   3. No autocorrelación: {'✅ CUMPLE' if 1.5 < dw_stat < 2.5 else '❌ NO CUMPLE'} (DW = {dw_stat:.4f})
   4. Homocedasticidad: {'✅ CUMPLE' if var_ratio < 2 else '❌ NO CUMPLE'} (ratio = {var_ratio:.4f})
""")

print("\n" + "━" * 80)
print("💡 INTERPRETACIÓN")
print("━" * 80)
print(f"""
   El modelo ANFIS fue entrenado con TODOS los datos disponibles ({best_result['n_samples']} muestras)
   sin partición, respetando la naturaleza secuencial de la serie temporal.
   
   Grid Search evaluó {len(results)} configuraciones diferentes.
   
   El modelo final utiliza {best_result['n_features']} características:
   {', '.join(best_result['feature_names'])}
   
   📊 Las 6 gráficas de diagnóstico están disponibles en la carpeta ANFIS/
""")

print("\n" + "═" * 80)
print(" " * 25 + "🎯 ANÁLISIS COMPLETADO")
print("═" * 80)


╔══════════════════════════════════════════════════════════════════════════════╗
║📊 RESUMEN FINAL DEL MODELO ANFIS 30 ║
╚══════════════════════════════════════════════════════════════════════════════╝

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🏆 MEJOR CONFIGURACIÓN
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   Parámetros del modelo:
   • Funciones de Membresía: 2
   • Lags AR: 2
   • Ventanas MA: 1
   • Regularización: 0.001
   • Tipo MF: gaussian
   • Features: ['y_t-1', 'y_t-2', 'm_2']


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📈 MÉTRICAS DE DESEMPEÑO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   • R² Score: 1.000000
   • MSE: 0.000000
   • MAE: 0.000000
   • RMSE: 0.000000
   • Muestras: 727


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ VERIFICACIÓN DE SUPUESTOS
━━━━━━━━━━━━━━━━━━━━━━━━━━━